In [1]:
# Youngwoo Kim
!hdfs dfs -D dfs.replication=1 -cp -f data/*.csv hdfs://nn:9000/

In [2]:
!hdfs dfs -du -h hdfs://nn:9000/*

328      328      hdfs://nn:9000/action_taken.csv
317      317      hdfs://nn:9000/agency.csv
521.0 K  521.0 K  hdfs://nn:9000/arid2017_to_lei_xref_csv.csv
311.6 K  311.6 K  hdfs://nn:9000/counties.csv
237      237      hdfs://nn:9000/denial_reason.csv
109      109      hdfs://nn:9000/edit_status.csv
180      180      hdfs://nn:9000/ethnicity.csv
166.8 M  166.8 M  hdfs://nn:9000/hdma-wi-2021.csv
41       41       hdfs://nn:9000/hoepa.csv
114      114      hdfs://nn:9000/lien_status.csv
65       65       hdfs://nn:9000/loan_purpose.csv
79       79       hdfs://nn:9000/loan_type.csv
129.6 K  129.6 K  hdfs://nn:9000/msamd.csv
122      122      hdfs://nn:9000/owner_occupancy.csv
92       92       hdfs://nn:9000/preapproval.csv
127      127      hdfs://nn:9000/property_type.csv
387      387      hdfs://nn:9000/purchaser_type.csv
252      252      hdfs://nn:9000/race.csv
144      144      hdfs://nn:9000/sex.csv
955      955      hdfs://nn:9000/states.csv
2.6 M    2.6 M    hdfs://nn:9000/trac

In [3]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName("cs544")
         .master("spark://boss:7077")
         .config("spark.executor.memory", "512M")
         .config("spark.sql.warehouse.dir", "hdfs://nn:9000/user/hive/warehouse")
         .enableHiveSupport()
         .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/07 23:52:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
banks_df = spark.read.options(header=True, inferSchema=True).csv("hdfs://nn:9000/arid2017_to_lei_xref_csv.csv")

In [5]:
type(banks_df)

pyspark.sql.dataframe.DataFrame

In [6]:
banks_df.rdd.take(3)

[Row(respondent_name='First National Bank', arid_2017='110004', lei_2018='5493003EW6T31TGECO83', lei_2019='5493003EW6T31TGECO83', lei_2020='5493003EW6T31TGECO83'),
 Row(respondent_name='First Mid Bank & Trust, National Association', arid_2017='110045', lei_2018='549300XOTES5TCS8T794', lei_2019='549300XOTES5TCS8T794', lei_2020='549300XOTES5TCS8T794'),
 Row(respondent_name='First Hope Bank, A National Banking Association', arid_2017='110118', lei_2018='5493003XLOX5FDT9R120', lei_2019='5493003XLOX5FDT9R120', lei_2020='5493003XLOX5FDT9R120')]

In [7]:
#q1
banks_df.rdd.filter(lambda x:"first" in x.respondent_name.lower()).count()

525

In [8]:
from pyspark.sql.functions import lower

In [9]:
#q2
banks_df.filter(lower(banks_df.respondent_name).like("%first%")).count()

525

In [10]:
#q3
banks_df.write.saveAsTable("banks", mode="overwrite")
spark.sql("""
            SELECT * FROM banks
            WHERE LOWER(respondent_name) LIKE '%first%'
            """).count()

23/11/07 23:52:46 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/11/07 23:52:46 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/11/07 23:52:51 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
23/11/07 23:52:51 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.22.0.5
23/11/07 23:52:52 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
23/11/07 23:52:58 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
23/11/07 23:52:58 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
23/11/07 23:52:58 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/11/07 23:52:58 W

525

In [11]:
loans_df = spark.read.options(header=True, inferSchema=True).csv("hdfs://nn:9000/hdma-wi-2021.csv")
loans_df.write.bucketBy(8, 'county_code').saveAsTable("loans", mode="overwrite")

23/11/07 23:53:13 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [12]:
list_of_tables = ["ethnicity", "race", "sex", "states", "counties", "tracts", "action_taken",
 "denial_reason", "loan_type", "loan_purpose", "preapproval", "property_type"]

In [13]:
for name in list_of_tables:
    spark.read.options(header=True, inferSchema=True).csv(f"hdfs://nn:9000/{name}.csv").createOrReplaceTempView(f"{name}")

In [14]:
#q4
tables = spark.sql("SHOW TABLES").toPandas()
result = {}
for i, row in tables.iterrows():
    result[row['tableName']] = row['isTemporary']
result

{'banks': False,
 'loans': False,
 'action_taken': True,
 'counties': True,
 'denial_reason': True,
 'ethnicity': True,
 'loan_purpose': True,
 'loan_type': True,
 'preapproval': True,
 'property_type': True,
 'race': True,
 'sex': True,
 'states': True,
 'tracts': True}

In [15]:
#q5
spark.sql("""
            SELECT lei_2020, respondent_name FROM banks
            INNER JOIN loans ON banks.lei_2020=loans.lei
            WHERE respondent_name = 'University of Wisconsin Credit Union'
            """).count()

19739

In [16]:
#q6
spark.sql("""
            SELECT lei_2020, respondent_name FROM banks
            INNER JOIN loans ON banks.lei_2020=loans.lei
            WHERE respondent_name = 'University of Wisconsin Credit Union'
            """).explain("formatted")
# respondent_name#64 and lei_2020#68
#

== Physical Plan ==
AdaptiveSparkPlan (8)
+- Project (7)
   +- BroadcastHashJoin Inner BuildLeft (6)
      :- BroadcastExchange (3)
      :  +- Filter (2)
      :     +- Scan parquet spark_catalog.default.banks (1)
      +- Filter (5)
         +- Scan parquet spark_catalog.default.loans (4)


(1) Scan parquet spark_catalog.default.banks
Output [2]: [respondent_name#64, lei_2020#68]
Batched: true
Location: InMemoryFileIndex [hdfs://nn:9000/user/hive/warehouse/banks]
PushedFilters: [IsNotNull(respondent_name), EqualTo(respondent_name,University of Wisconsin Credit Union), IsNotNull(lei_2020)]
ReadSchema: struct<respondent_name:string,lei_2020:string>

(2) Filter
Input [2]: [respondent_name#64, lei_2020#68]
Condition : ((isnotnull(respondent_name#64) AND (respondent_name#64 = University of Wisconsin Credit Union)) AND isnotnull(lei_2020#68))

(3) BroadcastExchange
Input [2]: [respondent_name#64, lei_2020#68]
Arguments: HashedRelationBroadcastMode(List(input[1, string, false]),false), [pla